In [ ]:
!pip install textattack
!pip install transformers
!pip install tensorflow-text
!pip install opacus
!pip install datasets
!pip install sentence-transformers

  Using cached importlib_metadata-3.10.1-py3-none-any.whl (14 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.11.3
    Uninstalling importlib-metadata-4.11.3:
      Successfully uninstalled importlib-metadata-4.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.


  Using cached importlib_metadata-4.11.3-py3-none-any.whl (18 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 3.10.1
    Uninstalling importlib-metadata-3.10.1:
      Successfully uninstalled importlib-metadata-3.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
konoha 4.6.5 requires importlib-metadata<4.0.0,>=3.7.0, but you have importlib-metadata 4.11.3 which is incompatible.


     |████████████████████████████████| 79 kB 5.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=deea1df7c6d90e70c75febd3ece6a4458914bb4f1163257055c2648bd572a6a9
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


# Prepare Data

In [ ]:
import tensorflow as tf

import tensorflow_datasets as tfds

ds, info = tfds.load('civil_comments', split='test', shuffle_files=True, with_info=True)
df = tfds.as_dataframe(ds, info)

Shuffling and writing examples to /root/tensorflow_datasets/civil_comments/CivilComments/1.0.0.incomplete49B3WW/civil_comments-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/civil_comments/CivilComments/1.0.0.incomplete49B3WW/civil_comments-validation.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/civil_comments/CivilComments/1.0.0.incomplete49B3WW/civil_comments-test.tfrecord


Dataset civil_comments downloaded and prepared to /root/tensorflow_datasets/civil_comments/CivilComments/1.0.0. Subsequent calls will reuse this data.


In [ ]:
df.columns

Index(['identity_attack', 'insult', 'obscene', 'severe_toxicity',
       'sexual_explicit', 'text', 'threat', 'toxicity'],
      dtype='object')

In [ ]:
df_mini = df[['text', 'toxicity']] 

def convert(data):
    parsed = data.decode('utf-8')
    return parsed

df_mini['text'] = df_mini['text'].apply(convert)

df_mini.to_csv('jigsaw_bias_test.csv', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
import pandas as pd
df_mini = pd.read_csv('jigsaw_bias_test.csv')
df_mini

,text,toxicity
0,"Science, Technology, Engineering, Math \n\nhtt...",0.000000
1,"Mostly safe, not counting the explosions, and ...",0.000000
2,"WSJ: Special Counsel, Congress Probing Money L...",0.166667
3,There really is no safe place on the planet to...,0.000000
4,"Skip, the problem with increasing EITC and oth...",0.000000
...,...,...
97315,That Murrikkka hasn't gone anywhere. It's just...,0.600000
97316,Only two years? Not a problem in the Nei when ...,0.300000
97317,Are you trying to say that black people are mo...,0.729730
97318,Saudi Arabia where this 'religion' originates ...,0.400000


In [ ]:
dataset = []
for i in range(len(df_mini)):
    text = df_mini.iloc[i]['text']
    label = int(df_mini.iloc[i]['toxicity'] > .5)
    dataset.append((text, label))

In [ ]:
dataset[-10:]

# Load Models

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pickle
import datasets
from torch.utils.data import DataLoader


BATCH_SIZE = 128

# needed for DP training
MAX_PHYSICAL_BATCH_SIZE = 64

ds = datasets.Dataset.from_pandas(df_mini)

train_dataloader = DataLoader(ds, batch_size=BATCH_SIZE)

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizer
import torch

from opacus import PrivacyEngine

EPOCHS = 1
delta_list = [5e-2, 1e-3, 1e-5]
NOISE_MULTIPLIER = 0.3
LEARNING_RATE = 1e-3
MAX_GRAD_NORM = 1

privacy_engine = PrivacyEngine()

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained('prajjwal1/bert-small')
model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, eps=1e-8)

model, optimizer, train_dataloader = privacy_engine.make_private(
    module=model,
    optimizer=optimizer,
    data_loader=train_dataloader,
    noise_multiplier=NOISE_MULTIPLIER,
    max_grad_norm=MAX_GRAD_NORM,
    poisson_sampling=False,
)
# model.load_state_dict(torch.load('model_state_dict.pt'))
# model.load_state_dict(torch.load('dp_model_state_dict.pt'))
model.load_state_dict(torch.load('dp_model_state_dict_noise_0.2.pt'))
# model.load_state_dict(torch.load('dp_model_state_dict_noise_0.25.pt'))
# model.load_state_dict(torch.load('dp_model_state_dict._noise_0.35.pt'))
# model.load_state_dict(torch.load('dp_model_state_dict_noise_0.3.pt'))
# model.load_state_dict(torch.load('dp_model_state_dict_noise_0.5.pt'))

model = model._module
model.eval()

/usr/local/lib/python3.7/dist-packages/opacus/privacy_engine.py:131: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "
Some weights of the model checkpoint at prajjwal1/bert-small were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. init

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 512, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, element

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained('prajjwal1/bert-small')
model.load_state_dict(torch.load('model_state_dict.pt'))
model.eval()

Some weights of the model checkpoint at prajjwal1/bert-small were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initi

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 512, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, element

In [ ]:
input_ids = tokenizer(['toxic sentence', 'The quick fox jumps over the excited and happy dog'],
                      truncation=True, padding='max_length', max_length=32, return_tensors='pt').input_ids
model(input_ids)['logits']

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1033: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


tensor([[-5.0831,  2.9641],
        [ 0.6450, -2.6431]], grad_fn=<AddmmBackward0>)

# Prepare Adversarial Attack

In [ ]:
from textattack import Attacker
from textattack.attack_recipes import PWWSRen2019, TextFoolerJin2019, A2TYoo2021
from textattack.datasets import HuggingFaceDataset
import textattack

# pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer)
# model_wrapper = HuggingFaceSentimentAnalysisPipelineWrapper(pipeline)
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)
# model_wrapper = textattack.models.wrappers.PyTorchModelWrapper(model, tokenizer)

# recipe = PWWSRen2019.build(model_wrapper)
recipe = TextFoolerJin2019.build(model_wrapper)
# recipe = A2TYoo2021.build(model_wrapper)

new_dataset = textattack.datasets.Dataset(dataset[:10000])

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


# Attacks

In [ ]:
attack_args = textattack.AttackArgs(num_examples=500)
attacker = Attacker(recipe, new_dataset, attack_args)
results = attacker.attack_dataset()

In [ ]:
# noise = .2

attack_args = textattack.AttackArgs(num_examples=500)
attacker = Attacker(recipe, new_dataset, attack_args)
results = attacker.attack_dataset()

In [ ]:
# noise = .25

attack_args = textattack.AttackArgs(num_examples=500)
attacker = Attacker(recipe, new_dataset, attack_args)
results = attacker.attack_dataset()

In [ ]:
# noise = .3

attack_args = textattack.AttackArgs(num_examples=500)
attacker = Attacker(recipe, new_dataset, attack_args)
results = attacker.attack_dataset()

In [ ]:
# noise = .35

attack_args = textattack.AttackArgs(num_examples=500)
attacker = Attacker(recipe, new_dataset, attack_args)
results = attacker.attack_dataset()

In [ ]:
# noise = .5

attack_args = textattack.AttackArgs(num_examples=500)
attacker = Attacker(recipe, new_dataset, attack_args)
results = attacker.attack_dataset()